# Tutorial 7: Conditional Schemas in RadPrompter

In this tutorial, we'll explore how to use **Conditional Schemas** in RadPrompter. This feature allows you to create schemas that are only processed when certain conditions are met based on previous schema responses. This is particularly useful for medical reports where follow-up questions depend on initial findings.

For example:
- First ask: "Is there pulmonary embolism?"
- If "Present": Ask follow-up questions about type, severity, etc.
- If "Absent": Skip the follow-up questions and use default values

## Installation

If you don't have `RadPrompter` installed, you can install it using pip:

```bash
pip install radprompter
```


## Understanding Conditional Schemas

Conditional schemas use the `depends_on` field to specify:
- **schema**: The name of the parent schema to check
- **condition**: The value that must be present to trigger this schema
- **default_value**: The value to use if the condition is not met

### Example TOML Structure:

```toml
[SCHEMAS.PulmonaryEmbolism]
variable_name = "Pulmonary Embolism"
type = "select"
options = ["Present", "Absent", "Indeterminate"]
hint = "Determine if there is evidence of pulmonary embolism..."

[SCHEMAS.EmbolismType]
variable_name = "Embolism Type"
type = "select"
options = ["Sub-segmental", "Segmental", "Main pulmonary artery", "Not applicable"]
depends_on = {schema = "Pulmonary Embolism", condition = "Present", default_value = "Not applicable"}
hint = "If pulmonary embolism is present, classify the type..."
```


In [ ]:
from radprompter import Prompt

prompt = Prompt('07_Conditional-Schemas.toml')
prompt


## Loading the Prompt

Let's examine the schemas to understand the conditional dependencies:

In [ ]:
# Print schema information
for i, schema in enumerate(prompt.schemas.schemas):
    print(f"Schema {i}: {schema['variable_name']}")
    if 'depends_on' in schema:
        depends_on = schema['depends_on']
        print(f"  → Depends on: {depends_on['schema']} = '{depends_on['condition']}'")
        print(f"  → Default value: '{depends_on['default_value']}'")
    else:
        print(f"  → Independent schema")
    print()

## Schema Processing Order

RadPrompter automatically determines the processing order based on dependencies:


In [ ]:
# Get the dependency order
processing_order = prompt.schemas.get_dependency_order()
print("Schema processing order:")
for i, schema_idx in enumerate(processing_order):
    schema = prompt.schemas.schemas[schema_idx]
    print(f"{i+1}. {schema['variable_name']} (index {schema_idx})")

## Client & Engine Setup

Set up the client and RadPrompter engine:


In [ ]:
from radprompter import RadPrompter, vLLMClient

client = vLLMClient(
    model="google/medgemma-4b-it",
    api_base="http://localhost:8000/v1",
    temperature=0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt,
    output_file="output_conditional_schemas.csv",
    concurrency=2,
    hide_blocks=False,
    use_pydantic=True
)

## Sample Reports

Let's load some sample reports to test the conditional logic:


In [ ]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

## Results Analysis

Let's examine the results to see how conditional schemas behaved:


In [ ]:
import pandas as pd

df = pd.read_csv("output_conditional_schemas.csv", index_col='index')
df

In [ ]:
# Analyze conditional schema behavior
print("Conditional Schema Analysis:")
print("=" * 50)

for index in df.index:
    row = df.loc[index]
    print(f"\nReport {index + 1}:")
    
    # Check PE status and dependent schemas
    pe_status = row['Pulmonary Embolism_response']
    print(f"  Pulmonary Embolism: {pe_status}")
    
    if pe_status == 'Present':
        print(f"  → Embolism Type: {row['Embolism Type_response']}")
        print("    (Should be actual classification)")
    else:
        print(f"  → Dependent schemas should show 'Not applicable'")
        print(f"    - Embolism Type: {row['Embolism Type_response']}")
    
    print(f"  File: {row['file_name']}")


## Key Benefits of Conditional Schemas

1. **Efficiency**: Only relevant follow-up questions are asked, reducing processing time and model calls
2. **Consistency**: Default values ensure consistent output format across all reports
3. **Medical Workflow**: Matches how radiologists actually read reports - initial assessment followed by detailed characterization if needed
4. **Clean Data**: Avoids confabulation in irrelevant fields by providing meaningful defaults
5. **Scalability**: Easy to add new conditional pathways without affecting existing logic

## Advanced Features

The conditional schema system supports:

- **Multiple conditions**: `condition = ["Present", "Indeterminate"]`
- **Complex dependencies**: Chain multiple levels of dependencies
- **Custom default values**: Any appropriate default for your use case
- **Multi-turn compatibility**: Works with single-turn and multi-turn prompts
- **Error handling**: Graceful handling of failed schema processing


## Example: Multi-level Dependencies

Here's an example of more complex conditional logic you could implement:

```toml
[SCHEMAS.FindingPresent]
variable_name = "Any Finding"
type = "select"
options = ["Present", "Absent"]
hint = "Are there any abnormal findings?"

[SCHEMAS.FindingType]
variable_name = "Finding Type"
type = "select"
options = ["Vascular", "Parenchymal", "Pleural", "Not applicable"]
depends_on = {schema = "Any Finding", condition = "Present", default_value = "Not applicable"}
hint = "Classify the type of finding..."

[SCHEMAS.VascularDetails]
variable_name = "Vascular Details"
type = "select"
options = ["Embolism", "Aneurysm", "Stenosis", "Not applicable"]
depends_on = {schema = "Finding Type", condition = "Vascular", default_value = "Not applicable"}
hint = "Specify the vascular pathology..."
```

This creates a three-level decision tree: Finding → Type → Specific Details.


## Save Processing Log

Finally, let's save the processing log for this session:


In [ ]:
engine.save_log("log_conditional_schemas.log")
print("Processing log saved to 'log_conditional_schemas.log'")

## Summary

In this tutorial, we've learned how to use conditional schemas in RadPrompter to create intelligent, context-aware questionnaires. Key takeaways:

1. **Conditional schemas** use the `depends_on` field to create branching logic
2. **Processing order** is automatically determined based on dependencies
3. **Default values** ensure consistent output when conditions aren't met
4. **Multi-turn support** works seamlessly with conditional logic
5. **Medical workflows** can be accurately modeled with this approach

This feature significantly improves the efficiency and accuracy of medical report processing by ensuring that only relevant questions are asked while maintaining consistent data structure.

For more complex use cases, you can chain multiple levels of dependencies and use multiple condition values to create sophisticated decision trees that mirror clinical reasoning patterns.
